In [4]:
# ! conda create -y -n milvus python=3.10

! pip install pymilvus==2.4.0
! pip install langchain
! pip install ibm-generative-ai
! pip install unstructured
! pip install huggingface-hub
! pip install sentence-transformers
! pip install ibm_watsonx_ai
! pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 139.6 kB/s eta 0:00:00a 0:00:03


In [5]:
import os
import requests
import json

from dotenv import load_dotenv

from langchain_community.document_loaders import UnstructuredURLLoader

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Milvus

from langchain.prompts import PromptTemplate
from langchain.callbacks.base import BaseCallbackHandler

# BAM
from genai.extensions.langchain import LangChainInterface
from genai.credentials import Credentials
from genai import Client

from genai.schema import TextGenerationParameters

from ibm_watsonx_ai.foundation_models import Embeddings

load_dotenv() 

False

In [6]:
token = "seu IAM API key IBM Cloud"
project_id = "aqui seu project_id do watsonx"

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter.split_text("gogo")

['gogo']

In [8]:
embedding_model = Embeddings(
            model_id="ibm/slate-125m-english-rtrvr",
            credentials={"api_key": token, "url": "https://us-south.ml.cloud.ibm.com"},
            project_id=project_id,
            params={
                "truncate_input_tokens": 512
            }
        )
embedding_model

In [9]:
# INITIALIZE A MILVUS COLLECTION WITH EMBEDDINGS FROM A SET OF DOCUMENTS

urls = [
    "https://www.tuasaude.com/receitas-de-crepioca-para-emagrecer/"
]

loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [10]:

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

vector_db = Milvus.from_documents(
    docs,
    embedding_model,
    drop_old=True,  # Drop the old Milvus collection if it exists
    connection_args={"host":"7aff4bea-4196-42e0-a9b7-bed85680916f.cie9agrw03kb77s3pr1g.lakehouse.appdomain.cloud", 
                     "port":"30141", "secure":True,
                     "user":"ibmlhapikey", "password": token},
)

In [11]:
# TEST COLLECTION INITIALISATION WITH A SIMPLE SIMILARITY SEARCH

query = "Como fazer tapioca?"
docs = vector_db.similarity_search(query)

docs

[Document(metadata={'source': 'https://www.tuasaude.com/receitas-de-crepioca-para-emagrecer/', 'pk': 452343555843950224}, page_content='Além disso, a crepioca é uma boa alternativa para pessoas com doença celíaca, já que não contém glúten, podendo substituir alimentos como pizzas, pães e bolos, por exemplo.\n\nInformação nutricional da crepioca\n\nA tabela a seguir traz a informação nutricional de 1 crepioca simples feita com 1 ovo e 2 colheres de sopa de tapioca:\n\nComponentes 1 crepioca simples Energia 146 calorias Proteínas 4,16 g Gorduras 3,48 g Carboidratos 24,55 g Fibras 0 g Vitamina A 45,2 mcg Luteína + zeaxantina 200,4 mcg\n\nPara se obter os benefícios da crepioca, é fundamental manter uma alimentação saudável e variada, além de fazer exercícios físicos regularmente.\n\nComo fazer a crepioca\n\nPara fazer a crepioca, basta misturar em uma tigela pequena 1 a 2 colheres de goma de tapioca, 1 ovo e sal. Em seguida, misturar todos os ingredientes com um garfo, até ficar bem homog

In [12]:
# SET LANGCHAIN INTERFACE FOR GEN AI

creds = Credentials(token,
                    api_endpoint="https://us-south.ml.cloud.ibm.com") # credentials object to access the LLM service


params = TextGenerationParameters(decoding_method="sample",temperature=0.7, top_p=1, top_k=50, min_new_tokens=0, max_new_tokens=200
                        )

model = LangChainInterface(model_id= 'ibm/granite-20b-multilingual', parameters=params, client=Client(credentials=creds))

prompt_template = (
            """[INST]
            <|system|>
            Aqui estão suas instruções:
            1. Você receberá um contexto que deverá ser usado para responder às perguntas dos usuários. O contexto começará com \"BEGIN\" e terminará com \"END\".
            2. Você deve gerar a próxima resposta utilizando informações disponíveis no contexto.
            4. Não use nenhum outro conhecimento.
            5. Especifique as frases no contexto que justificam a sua resposta.
                BEGIN {context} END
            [/INST]
            {question}
            <|assistant|>
            """
        )

customprompt = PromptTemplate(
            template=prompt_template, input_variables=["context", "question"]
        )

In [13]:
model

LangChainInterface(client=<genai.client.Client object at 0x104f46f30>, model_id='ibm/granite-20b-multilingual', parameters=TextGenerationParameters(beam_width=None, decoding_method='sample', include_stop_sequence=None, length_penalty=None, max_new_tokens=200, min_new_tokens=0, random_seed=None, repetition_penalty=None, return_options=None, stop_sequences=None, temperature=0.7, time_limit=None, top_k=50, top_p=1.0, truncate_input_tokens=None, typical_p=None))

In [12]:
def append_docs(docs):
    context = ''
    for doc in docs:
        context += doc.page_content
    return context

def get_wkc_token_on_cloud(apikey):
    body = {"grant_type": "urn:ibm:params:oauth:grant-type:apikey",
            "apikey": apikey}

    token_request = requests.post('https://iam.cloud.ibm.com/identity/token',
                                  data=body, headers={'content-type': 'application/x-www-form-urlencoded'})
    try:
        token_request.raise_for_status()
    except requests.exceptions.HTTPError as e:
        raise e  # return code not equals to 200

    return json.loads(token_request.text)["access_token"]


In [13]:
def qa(question):
    docs = vector_db.similarity_search(query)

    print(str(len(docs))+" nearest embeddings found;")
    
    context = append_docs(docs=docs)
    prompt = customprompt.format(context=context, question=question)

    barear_token = get_wkc_token_on_cloud(token)

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"

    body = {
        "input": prompt,
        "parameters": {
            "decoding_method": "greedy",
            "max_new_tokens": 900,
        },
        "model_id": "ibm/granite-20b-multilingual",
        "project_id": project_id
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(barear_token)
    }

    response = requests.post(
        url,
        headers=headers,
        json=body
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))
    
    data = response.json()
    
    return data['results'][0]['generated_text']

In [14]:
print(qa("Como fazer crepioca?"))

4 nearest embeddings found;
 Aqui está uma receita simples de crepioca:
             Ingredientes:
             - 1 ovo;
             - 2 colheres (de sopa) de goma de tapioca;
             - Sal a gosto.

1. Em um recipiente fundo, bater bem o ovo com o garfo.

2. Adicionar a goma de tapioca e o sal, misturando bem.

3. Despejar a mistura em uma frigideira antiaderente preaquecida no fogo.

4. Deixar assar por cerca de 2 a 3 minutos em cada lado ou até a crepioca ficar dourada.

5. Transferir para um prato e rechear com o queijo, frango, presunto, requeijão, frutas, entre outros ingredientes de sua preferência.

A crepioca é uma ótima opção para o café da manhã, lanche da tarde, almoço ou jantar. É rápida e fácil de fazer, além de ser uma ótima fonte de proteínas, vitaminas e carboidratos. Você pode rechear com diversos ingredientes, de acordo com sua preferência e dieta.


In [15]:
print(qa("Crepioca é calórico?"))

4 nearest embeddings found;
 A crepioca é uma opção de alimento que pode ser consumido sem medo, já que é considerada um alimento saudável. Ela é rica em proteínas, carboidratos e gorduras boas, que fornecem energia e ajudam no ganho de massa muscular.

Além disso, a crepioca pode ser recheada com diversos ingredientes saudáveis, como frutas, queijo, aveia e até mesmo legumes, tornando-a uma opção nutritiva e saborosa.

Portanto, a crepioca não é um alimento calórico, mas sim uma opção de alimento nutritivo e saboroso que pode fazer parte de uma dieta equilibrada e saudável.

